# 1. simple sequential model: MLP for minist

In [25]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [26]:
# Xtr, Ytr = mnist.train.next_batch(5000)
# Xte, Yte = mnist.test.next_batch(200)
Xtr, Ytr = mnist.train.images, mnist.train.labels
Xte, Yte = mnist.test.images, mnist.test.labels
print type(Xtr)
print Xtr.shape

<type 'numpy.ndarray'>
(55000, 784)


## sklearn-style interface

In [5]:
from keras.models import Sequential
model = Sequential() # a linear stack of layers
from keras.layers import Dense, Activation
# Stacking layers is as easy as .add()
model.add(Dense(output_dim=256, input_dim=784, batch_input_shape=[None, 784]))
model.add(Activation("relu"))
model.add(Dense(output_dim=256, input_dim=256))
model.add(Activation("relu"))
model.add(Dense(output_dim=10))
model.add(Activation("softmax"))
# Once your model looks good, configure its learning process with .compile()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
model.fit(Xtr, Ytr, nb_epoch=5, batch_size=100)
print model.evaluate(Xte, Yte, batch_size=100)

Epoch 1/5
55000/55000 [==============================] - 3s - loss: 0.2548 - acc: 0.9267     
Epoch 2/5
55000/55000 [==============================] - 3s - loss: 0.0966 - acc: 0.9702     
Epoch 3/5
55000/55000 [==============================] - 3s - loss: 0.0628 - acc: 0.9799     
Epoch 4/5
55000/55000 [==============================] - 3s - loss: 0.0427 - acc: 0.9869     
Epoch 5/5
 9700/10000 [============================>.] - ETA: 0s[0.071162071719591047, 0.97690000772476193]


## training in batches

In [7]:
training_epochs = 5
batch_size = 100
display_step = 1

model.reset_states() # doesn't work?
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for epoch in xrange(1, training_epochs+1):
    total_batch = int(mnist.train.num_examples / batch_size)
    avg_cost, avg_acc = 0, 0
    for i in xrange(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)
        c,a = model.train_on_batch(xs, ys)
        avg_cost += c/total_batch
        avg_acc += a/total_batch
    if epoch % display_step == 0: 
        print 'epoch %d, cost=%.9f, acc=%.9f' %(epoch, avg_cost, avg_acc)
print 'finish!'
print model.evaluate(Xte, Yte, batch_size=100)

epoch 1, cost=0.034733955, acc=0.988036372
epoch 2, cost=0.021587116, acc=0.993200006
epoch 3, cost=0.018467821, acc=0.993981824
epoch 4, cost=0.015665767, acc=0.994654550
epoch 5, cost=0.011712108, acc=0.996127276
finish!
 9500/10000 [===========================>..] - ETA: 0s[0.088480864168632253, 0.97980000913143162]


# CNN on minist

In [9]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [10]:
# parameters
batch_sz = 128
n_classes = 10
n_epoch = 12
img_rows, img_cols = 28, 28
n_filters = 32
pool_sz = (2,2)
kernel_sz = (3,3)

In [23]:
(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

In [15]:
print type(Xtrain)
print Xtrain.shape

<type 'numpy.ndarray'>
(60000, 28, 28)


Data propressing

**note**: 
The image ordering is NOT the same for tf and th! 

In [13]:
print K.image_dim_ordering()

tf


In [28]:
Xtrain = Xtrain.reshape( (-1, img_rows, img_cols, 1) )
Xtest = Xtest.reshape( (-1, img_rows, img_cols, 1) )
input_shape = (img_rows, img_cols, 1)
print Xtrain.shape, Xtest.shape

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [32]:
# the keras mnist dataset is int, turn to float
Xtrain = Xtrain.astype('float')
Xtest = Xtest.astype('float')

In [33]:
# normalize data
Xtrain /= 255.0
Xtest /=255.0

In [35]:
print Ytrain

[5 0 4 ..., 5 6 8]


In [36]:
# do one-hot encoding
Ytrain = np_utils.to_categorical(Ytrain, n_classes)
Ytest = np_utils.to_categorical(Ytest, n_classes)

In [37]:
model = Sequential()
model.add( Convolution2D(n_filters, kernel_sz[0], kernel_sz[1],
                        border_mode='valid',
                        input_shape=input_shape) )
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=pool_sz))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(output_dim=128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes))
model.add(Activation('softmax'))

In [38]:
model.compile(loss='categorical_crossentropy',
             optimizer='adadelta',
             metrics=['accuracy'])

In [40]:
model.fit(Xtrain, Ytrain,
         batch_size=batch_sz,
         nb_epoch=n_epoch,
         verbose=1,
         validation_data=(Xtest, Ytest) )

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 25s - loss: 0.4458 - acc: 0.8693 - val_loss: 0.1502 - val_acc: 0.9553
Epoch 2/12
60000/60000 [==============================] - 24s - loss: 0.1995 - acc: 0.9412 - val_loss: 0.1003 - val_acc: 0.9720
Epoch 3/12
60000/60000 [==============================] - 25s - loss: 0.1530 - acc: 0.9556 - val_loss: 0.0849 - val_acc: 0.9741
Epoch 4/12
60000/60000 [==============================] - 24s - loss: 0.1288 - acc: 0.9629 - val_loss: 0.0692 - val_acc: 0.9782
Epoch 5/12
60000/60000 [==============================] - 23s - loss: 0.1125 - acc: 0.9672 - val_loss: 0.0638 - val_acc: 0.9801
Epoch 6/12
60000/60000 [==============================] - 24s - loss: 0.1014 - acc: 0.9697 - val_loss: 0.0570 - val_acc: 0.9816
Epoch 7/12
60000/60000 [==============================] - 24s - loss: 0.0948 - acc: 0.9720 - val_loss: 0.0535 - val_acc: 0.9825
Epoch 8/12
60000/60000 [==============================

In [41]:
c,a = model.evaluate(Xtest, Ytest, verbose=0)
print c,a

0.0448687467696 0.9842
